In [1]:
import numpy
import pandas
import morphio
import conntility
import os

from scipy.spatial import distance

root_morph = "/gpfs/bbp.cscs.ch/project/proj3/projects-data/2023.12.18-microns-skeletonization"
conn_fn = "/gpfs/bbp.cscs.ch/home/reimann/data/mm3/microns_mm3_connectome.h5"

root_morph_out = "./translated"
root_sonata_out = "./sonata"
sonata_edges = "edges.h5"
sonata_nodes = "nodes.h5"
node_population = "default"
edge_population = "default"

M = conntility.ConnectivityMatrix.from_h5(conn_fn, "full")

# Read contents of root_morph.
Finds all swc files in Marwan's morphology release and creates a Series where the index is the "pt_root_id" and the value is the path to the morphology file

In [2]:
subdirs = [_x for _x in os.listdir(root_morph) if _x.startswith("2023")]

def dir_to_series(root):
    fns = [_x for _x in os.listdir(root) if 
           os.path.isfile(os.path.join(root, _x)) and 
           os.path.splitext(_x)[1] == ".swc"]
    return pandas.Series([
        os.path.join(root, _x) for _x in fns
    ], index=[
        int(os.path.splitext(_x)[0]) for _x in fns
    ])

morph_series = dir_to_series(os.path.join(root_morph, subdirs[0]))

# Generate subnetwork
Create the subgraph of M that contains all nodes that are contained in the release, plus the edges between them

In [3]:
lookup = M._vertex_properties["pt_root_id"].reset_index().set_index("pt_root_id")["index"]
S = M.subpopulation(lookup[morph_series.index].values)

# Translate morphologies
This is not strictly necessary, but we prefer morphologies that are centered with their soma at the origin.
The location of the neuron is then given as a node property.

Here, we create copies of all morphologies that are translated by the corresponding node location in M.

In [4]:
def translate_morphology(m, offset, out_fn):
    mm = m.as_mutable()
    offset = offset.reshape((1, -1))
    mm.soma.points = mm.soma.points - offset
    for sec in mm.iter():
        sec.points = sec.points - offset

    m2 = mm.as_immutable()
    mm.write(out_fn)
    return m2

morph_series_tl = {} # As morph_series, but for the translated morphology files

for i, nrn in S.vertices.iterrows():
    offset = (nrn[["x_nm", "y_nm", "z_nm"]] / 1000).values.reshape((1, -1))
    m = morphio.Morphology(morph_series[nrn["pt_root_id"]])
    _, fn_base = os.path.split(morph_series[nrn["pt_root_id"]])
    fn_out = os.path.join(root_morph_out, fn_base)
    m2 = translate_morphology(m, offset, fn_out)
    morph_series_tl[nrn["pt_root_id"]] = fn_out

morph_series_tl = pandas.DataFrame(morph_series_tl, index=["path"]).transpose()["path"]

# Create Sonata-compatible synapse DataFrame
This is the only really complex step.
While M.edges is already a DataFrame of synapse properties, we will have to make it Sonata compatible.

That comprises renaming some columns, moving from nm to um. But also looking up the section and segment ids where all synapses are placed. So, we do a bit of 3d geometry here.

In [5]:
def morphology(nrn):
    m = morphio.Morphology(morph_series_tl[nrn["pt_root_id"]])
    return m

def point_segment_distances(sec_pts, syn_locs):
    dist_start = distance.cdist(syn_locs, sec_pts[:-1])
    dist_end = distance.cdist(syn_locs, sec_pts[1:])

    d_sec = numpy.diff(sec_pts, axis=0)
    seg_len = numpy.linalg.norm(d_sec, axis=1, keepdims=True)
    d_sec = d_sec / seg_len
    d_syn = syn_locs.reshape((-1, 1, 3)) - sec_pts[:-1].reshape((1, -1, 3)) # syns x segs x 3
    dist_syn = numpy.linalg.norm(d_syn, axis=-1, keepdims=True)
    d_syn_norm = d_syn / dist_syn

    cos_angle = (d_syn_norm * d_sec).sum(axis=-1)
    pw_dist = numpy.sin(numpy.arccos(cos_angle)) * dist_syn[:, :, 0]

    proj_dist = cos_angle * dist_syn[:, : , 0]
    pw_dist[proj_dist < 0] = dist_start[proj_dist < 0]
    pw_dist[proj_dist > seg_len.transpose()] = dist_end[proj_dist > seg_len.transpose()]
    
    seg_offsets = numpy.maximum(numpy.minimum(proj_dist, seg_len.transpose()), 0)
    
    return pw_dist, seg_offsets

def point_sections_distances(m2, syn_locs):
    all_pw_dist, all_offsets = zip(*[
        point_segment_distances(_sec.points, syn_locs)
        for _sec in m2.sections
    ])
    return numpy.hstack(all_pw_dist), numpy.hstack(all_offsets)

def find_synapse_location_on_morphology(m2, syn_locs, soma_epsilon=1.0):
    # Assumes morphology is centered on soma!
    dist_from_soma = numpy.linalg.norm(syn_locs, axis=1)
    is_on_soma = dist_from_soma < (soma_epsilon + m2.soma.diameters / 2)
    
    n_per_sec = numpy.diff(m2.section_offsets) - 1
    
    sec_lo = numpy.hstack([i * numpy.ones(n, dtype=int)
                           for i, n in enumerate(n_per_sec)])
    
    all_pw_dist, all_offsets = point_sections_distances(m2, syn_locs)
    D = numpy.min(all_pw_dist, axis=1)
    idxx = numpy.argmin(all_pw_dist, axis=1)
    
    res_sec = sec_lo[idxx]
    res_seg = idxx - numpy.cumsum(numpy.hstack([0, n_per_sec]))[res_sec]
    res_off = all_offsets[numpy.arange(all_offsets.shape[0]), idxx]
    
    res_sec[is_on_soma] = -1
    res_seg[is_on_soma] = 0
    res_off[is_on_soma] = 0
    D[is_on_soma] = numpy.maximum(dist_from_soma[is_on_soma] - m2.soma.diameters / 2, 0)
    
    return res_sec, res_seg, res_off, D

def base_synapse_properties_df(m2, syn_locs, soma_epsilon=1.0):
    res_sec, res_seg, res_off, D = find_synapse_location_on_morphology(m2, syn_locs, soma_epsilon)
    sec_types = m2.section_types[res_sec]
    sec_types[res_sec == -1] = 1 # SOMA
    return pandas.DataFrame({
            "afferent_section_id": res_sec,
            "afferent_segment_id": res_seg,
            "afferent_segment_offset": res_off,
            "afferent_section_type": sec_types,
            "spine_length": D
        })
    
def synapse_properties_df(S, nrn, lst_additional_props):
    m = morphology(nrn)
    Ssub = S.filter("pt_root_id", side="col").eq(nrn["pt_root_id"])
    syn_locs = Ssub.edges[["delta_x_nm", "delta_y_nm", "delta_z_nm"]].values / 1000
    props = base_synapse_properties_df(m, syn_locs)
    
    xyz = Ssub.edges[["x_nm", "y_nm", "z_nm"]].reset_index(drop=True) / 1000
    props = pandas.concat([
        xyz.rename(columns={"x_nm": "afferent_center_x", "y_nm": "afferent_center_y", "z_nm": "afferent_center_z"}),
        props,
        Ssub.edges[lst_additional_props].reset_index(drop=True)
    ], axis=1)
    
    
    idxx = pandas.DataFrame({
        "source_node_id": Ssub._edge_indices.values[:, 0],
        "target_node_id": Ssub._edge_indices.values[:, 1]
    })
    return props, idxx

In [6]:
props, idxx = zip(*[synapse_properties_df(S, nrn, ["size"]) for _, nrn in S.vertices.iterrows()])
props = pandas.concat(props, axis=0).reset_index(drop=True)
idxx = pandas.concat(idxx, axis=0).reset_index(drop=True)

# Write EdgePopulation

In [9]:
import h5py
from libsonata import EdgePopulation

if not os.path.exists(root_sonata_out):
    os.makedirs(root_sonata_out)

output_path = os.path.join(root_sonata_out, sonata_edges)
population = edge_population

with h5py.File(output_path, 'w') as h5:
    grp = h5.create_group('/edges/{0}'.format(population))
    for _col in idxx.columns:
        grp.create_dataset(_col, data=idxx[_col].values.astype(numpy.int64))
    
    prop_grp = grp.create_group("0")
    for _col in props.columns:
        prop_grp.create_dataset(_col, data=props[_col].values)
        
EdgePopulation.write_indices(
        output_path,
        population,
        len(S),
        len(S)
    )


# Write NodePopulation
Similarly, for the node population we have to rename some columns and move from nm to um.

In [10]:
from voxcell import CellCollection

relevant_cols = ["pt_root_id", "cell_type", "tentative_region"]
tl = {"tentative_region": "region"}

position_cols = ["x_nm", "y_nm", "z_nm"]
tl_pos = {"x_nm": "x", "y_nm": "y", "z_nm": "z"}

# morphology filename w/o root and extension
_m_s = morph_series_tl.apply(lambda _x: os.path.splitext(os.path.split(_x)[1])[0])
_m_s.name = "morphology"

def append_quaternions(df_in):
    df_q = pandas.DataFrame({
        "orientation_w": numpy.ones(len(df_in)),
        "orientation_x": numpy.zeros(len(df_in)),
        "orientation_y": numpy.zeros(len(df_in)),
        "orientation_z": numpy.zeros(len(df_in))
    }, index=df_in.index)
    return pandas.concat([df_in, df_q], axis=1)

df = append_quaternions(
    pandas.concat([S.vertices[relevant_cols].rename(columns=tl),
                   S.vertices[position_cols].rename(columns=tl_pos) / 1000,
                   _m_s[S.vertices["pt_root_id"]].reset_index(drop=True)],
             axis=1)
)
df.index = numpy.arange(1, len(df) + 1)

coll = CellCollection.from_dataframe(df)
coll.save_sonata(os.path.join(root_sonata_out, sonata_nodes))
